# Gerekli Kütüphaneler

In [2]:
import numpy as np
import pandas as pd
import re
import string
import spacy
import requests
from nltk.corpus import stopwords
from collections import Counter
from nltk.tokenize import word_tokenize
from datetime import datetime

import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(format = '%(asctime)s %(message)s',
                    datefmt = '%m/%d/%Y %I:%M:%S %p',
                    filename = 'logs/4_feature_engineering.log',
                    level=logging.DEBUG)

import nltk
#nltk.download('popular')

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

# Veriyi yükleyelim

In [18]:
df_ = pd.read_csv('data/cleaned.csv', index_col=0, na_values='?')
logging.debug('Temizlenmiş veri seti yüklendi.')
df = df_.copy()
df.head()

,Dizi Linki,Dizi Adı,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,Görüntü yönetmeni,Gösterim süresi,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Tarih,Youtube,Facebook,Twitter,Instagram,Durumu
0,https://tr.wikipedia.org//wiki/Kaynanalar,Kaynanalar,Komedi,NaN,NaN,"Tekin Akmansoy,Arzu Akmansoy","Tekin Akmansoy,Leman Çıdamlı,Defne Yalnız,Sevda Aydan,Ege Aydan,Münir Caner,Sermin Hürmeriç,Sezai Aydın",NaN,14.00,314,NaN,NaN,NaN,NaN,NaN,"TRT 1,Kanal D",NaN,NaN,19 Mayıs 1974 - 24 Ocak 2005,1974.00,0,0,0,0,Sona erdi
1,"https://tr.wikipedia.org//wiki/A%C5%9Fk-%C4%B1_Memnu_(dizi,_1975)",Aşk-ı Memnu,"Aile,Dram",NaN,Halit Refiğ,Halit Refiğ,"Suna Keskin,Müjde Ar,Şükran Güngör,Neriman Köksal,Itır Esen",NaN,1.00,6,NaN,İstanbul,NaN,NaN,NaN,TRT 1,NaN,NaN,19 Nisan 1975 - 24 Mayıs 1975,1975.00,0,0,0,0,Sona erdi
2,NaN,Kartallar Yüksek Uçar,Polisiye,NaN,Atilla İlhan,Hüseyin Karakaş,"Selda Alkor,Hüseyin Peyda,Serap Aksoy,Necmettin Çobanoğlu,Can Gürzap",NaN,1.00,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26 Kasım 1983,1983.00,0,0,0,0,Yayından kaldırıldı
3,https://tr.wikipedia.org//wiki/%C3%9C%C3%A7_%C4%B0stanbul_(dizi),Üç İstanbul,Dram,NaN,Bülent Oran,Feyzi Tuna,"Ayda Aksel,Gülsen Tuncer,Celile Toyon,Burçin Oraloğlu,Nilgün Akçaoğlu",Timur Selçuk,1.00,NaN,Oskay Alptürk,NaN,Muzaffer Turan,NaN,Ercan Film,TRT 1,NaN,NaN,1983,1983.00,0,0,0,0,Sona erdi
4,"https://tr.wikipedia.org//wiki/K%C3%BC%C3%A7%C3%BCk_A%C4%9Fa_(dizi,_1984)",Küçük Ağa,"Dram,Aksiyon,Tarihî",NaN,Tarık Buğra,Yücel Çakmaklı,"Çetin Tekindor,Aydan Şener,Erol Taş,Fikret Hakan",NaN,1.00,8,İlksen Bektaş,NaN,Sineray,58.00,TRT,TRT 1,NaN,NaN,"25 Mart 1984,(,1984-03-25,),[1],-,13 Mayıs 1984,(,1984-05-13,),[2]",1984.00,0,0,0,0,Sona erdi


# Boş değerleri dolduralım

In [19]:
df = df[~df['Durumu'].isna()]

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1252 entries, 0 to 1255
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Dizi Linki         924 non-null    object 
 1   Dizi Adı           1252 non-null   object 
 2   Tür                1252 non-null   object 
 3   Uyarlama           85 non-null     object 
 4   Senarist           1118 non-null   object 
 5   Yönetmen           1211 non-null   object 
 6   Başrol             1250 non-null   object 
 7   Besteci            514 non-null    object 
 8   Sezon sayısı       1208 non-null   float64
 9   Bölüm sayısı       1082 non-null   object 
 10  Yapımcı            1070 non-null   object 
 11  Mekân              765 non-null    object 
 12  Görüntü yönetmeni  430 non-null    object 
 13  Gösterim süresi    912 non-null    float64
 14  Yapım şirketi      930 non-null    object 
 15  Kanal              1017 non-null   object 
 16  Platform           123 non-nu

In [21]:
df['Sezon sayısı'].fillna(1.0, inplace=True)
logging.debug('Sezon sayısı boş olanlar 1 ile dolduruldu.')

In [22]:
df['Uyarlama'].fillna('Uyarlama Değil', inplace=True)

In [23]:
df['Gösterim süresi'].fillna(method='bfill', inplace=True)
logging.debug('Gösterim süresi boş olanlar bir önceki sütundaki ile dolduruldu.')

/tmp/ipykernel_2956/1122797201.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Gösterim süresi'].fillna(method='bfill', inplace=True)


In [24]:
df['Kanal'].fillna('Diğer', inplace=True)

In [25]:
df['Mekân'].fillna('İstanbul', inplace=True)

In [26]:
df.isnull().sum()

Dizi Linki            328
Dizi Adı                0
Tür                     0
Uyarlama                0
Senarist              134
Yönetmen               41
Başrol                  2
Besteci               738
Sezon sayısı            0
Bölüm sayısı          170
Yapımcı               182
Mekân                   0
Görüntü yönetmeni     822
Gösterim süresi         0
Yapım şirketi         322
Kanal                   0
Platform             1129
Ses formatı           446
Yayın tarihi            0
Tarih                   0
Youtube                 0
Facebook                0
Twitter                 0
Instagram               0
Durumu                  0
dtype: int64

In [27]:
df.fillna('?', inplace=True)

In [28]:
df.to_csv('data/cleaned_filled.csv')
logging.debug('Doldurulan veri seti kaydedildi.')

# Özellik Çıkarımı

In [29]:
feature_df = pd.DataFrame()
logging.debug('Özellikleri çıkarmak için ayrı bir veri seti oluşturuldu.')

In [30]:
feature_df['Tarih'] = df['Tarih']
logging.debug('Tarih özelliği oluşturuldu.')

In [31]:
feature_df['Dizi Adı'] = df['Dizi Adı']
logging.debug('Dizi adı özelliği oluşturuldu.')

In [32]:
feature_df['Dizi Adı Uzunluğu'] = df['Dizi Adı'].apply(lambda x: len(x))
logging.debug('Dizi adı uzunluğu özelliği oluşturuldu.')

In [33]:
logging.debug('En kısa dizi adı uzunluğuna sahip dizi bulundu.')
df[feature_df['Dizi Adı Uzunluğu'] == feature_df['Dizi Adı Uzunluğu'].min()]

,Dizi Linki,Dizi Adı,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,Görüntü yönetmeni,Gösterim süresi,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Tarih,Youtube,Facebook,Twitter,Instagram,Durumu
867,https://tr.wikipedia.org//wiki/Fi_(dizi),Fi,"Aşk,Gerilim","Azra Kohen,'in,Fi,,,Çi,,,Pi,isimli kitap serisinden",Nükhet Bıçakçı,Mert Baykal,"Ozan Güven,Serenay Sarıkaya,Mehmet Günsür,Berrak Tüzünataç,Büşra Develi,Tülay Günal,Osman Sonant",Cem Öget,2.00,22,"Kerem Çatay,Pelin Diştaş Yaşaroğlu",İstanbul,?,60.00,Ay Yapım,Diğer,puhutv,Stereo,"31 Mart 2017,(,2017-03-31,),[1],-,9 Mart 2018,(,2018-03-09,)",2017.00,0,0,1,1,Sona erdi


In [34]:
logging.debug('En uzun dizi adına sahip olan dizi bulundu.')
df[feature_df['Dizi Adı Uzunluğu'] == feature_df['Dizi Adı Uzunluğu'].max()]

,Dizi Linki,Dizi Adı,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,Görüntü yönetmeni,Gösterim süresi,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Tarih,Youtube,Facebook,Twitter,Instagram,Durumu
534,https://tr.wikipedia.org//wiki/Esir_%C5%9Eehrin_G%C3%B6zya%C5%9Flar%C4%B1_-_Bir_Ferhat_ile_%C5%9Eirin_Hikayesi,Esir Şehrin Gözyaşları - Bir Ferhat ile Şirin Hikayesi,Dram,Uyarlama Değil,Sevgi Yılmaz,Çağatay Tosun,"Yiğit Özşener,Sedef Avcı",?,1.00,5,Ata Türkoğlu,İstanbul,?,90.00,Koliba Film,FOX,?,Stereo,28 Eylül 2012 - 31 Ekim 2012,2012.00,0,0,0,0,Sona erdi


# Dizi Adındaki Toplam Kelime Sayısı

In [35]:
def find_word_count(text):
    text = word_tokenize(text)
    return len(text)

In [36]:
feature_df['Dizi Adındaki Kelime Sayısı'] = df['Dizi Adı'].apply(find_word_count)
logging.debug('Dizi adındaki toplam kelime sayısı özelliği oluşturuldu.')

In [37]:
logging.debug('Adında en az kelime bulunan dizi bulundu.')
df[feature_df['Dizi Adındaki Kelime Sayısı'] == feature_df['Dizi Adındaki Kelime Sayısı'].min()]

,Dizi Linki,Dizi Adı,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,Görüntü yönetmeni,Gösterim süresi,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Tarih,Youtube,Facebook,Twitter,Instagram,Durumu
0,https://tr.wikipedia.org//wiki/Kaynanalar,Kaynanalar,Komedi,Uyarlama Değil,?,"Tekin Akmansoy,Arzu Akmansoy","Tekin Akmansoy,Leman Çıdamlı,Defne Yalnız,Sevda Aydan,Ege Aydan,Münir Caner,Sermin Hürmeriç,Sezai Aydın",?,14.00,314,?,İstanbul,?,58.00,?,"TRT 1,Kanal D",?,?,19 Mayıs 1974 - 24 Ocak 2005,1974.00,0,0,0,0,Sona erdi
5,https://tr.wikipedia.org//wiki/Ac%C4%B1mak_(dizi),Acımak,Dram,Uyarlama Değil,Orhan Aksoy,Orhan Aksoy,"Ayşegül Aldinç,Ediz Hun,Berhan Şimşek,Nazan Ayas,Oktar Durukan,Hulusi Kentmen,Muhip Arcıman",?,1.00,7,Hilmi Akyalçın,İstanbul,?,215.00,TRT,TRT 1,?,?,"22 Aralık 1985,(,1985-12-22,),[1],-,2 Şubat 1986,(,1986-02-02,),[2]",1985.00,0,0,0,0,Sona erdi
9,"https://tr.wikipedia.org//wiki/%C3%87al%C4%B1ku%C5%9Fu_(dizi,_1986)",Çalıkuşu,Dram,Çalıkuşu,Osman F. Seden,Osman F. Seden,"Aydan Şener,Kenan Kalav,Sadri Alışık,Mine Çayıroğlu,Eşref Kolçak,Hayati Hamzaoğlu,Kaya Akarsu,Tomris Oğuzalp,Savaş Başar,Seray Gözler,Tilbe Saran,Cengiz Sezici,Sümer Tilmaç,Yalçın Avşar,Ali Şen,Mübeccel Vardar",Esin Engin,1.00,7,"TRT,adına,Osman F. Seden","İstanbul,Bolu,İzmir,Bursa",?,45.00,?,TRT 1,?,?,20 Nisan 1986 - 1 Haziran 1986,1986.00,0,0,0,0,Sona erdi
14,https://tr.wikipedia.org//wiki/U%C4%9Furlugiller,Uğurlugiller,Komedi,Uyarlama Değil,"İsmet Kür,Turgut Özakman,Ülker Köksal","Gürol Gökçe,Uğur Erkır,Aram Gülyüz","Yıldız Kenter,Şükran Güngör,Tevfik Gelenbe,Gül Onat,Mehmet Birkiye,Mübeccel Vardar,Ayhan Kavas,Mustafa Alabora,Savaş Dinçel",?,4.00,68,Uğur Erkır,İstanbul,?,25.00,STR,TRT 1,?,?,6 Temmuz 1988 - 28 Ocak 1991,1988.00,0,0,0,0,Sona erdi
16,https://tr.wikipedia.org//wiki/Gen%C3%A7ler_(dizi),Gençler,"Dram,Gençlik,Komedi",Uyarlama Değil,Sıtkı Tekmen,Adem Kılıç,"Ziya Kürküt,Fikret Kuşkan,Oktay Kaynarca,Bilge Parlak,Melike Altunbaran,Zeynep Özden Ayyıldız,Çetin Etili",?,1.00,?,?,İstanbul,?,50.00,TRT,TRT 1,?,?,1989 - 1990,1989.00,0,0,0,0,Sona erdi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,https://tr.wikipedia.org//wiki/Safir_(dizi),Safir,Dram,Uyarlama Değil,"Burcu Över,Gül Abus Semerci,Nalan Merter Savaş,Onur Koçal,",Semih Bağcı,"İlhan Şen,Özge Yağız,Burak Berkay Akgül,Gizem Karaca","Ender Gündüzlü,Metin Arıgül,Can Sanıbelli",1.00,16,"Fatih Aksoy,Mehmet Yiğit Alp","Kapadokya,,,Nevşehir",?,135.00,NTC Medya,atv,?,Stereo,4 Eylül 2023 - günümüz,2023.00,1,1,1,1,Devam ediyor
1236,https://tr.wikipedia.org//wiki/Terzi_(dizi),Terzi,Dram,Uyarlama Değil,"Rana Mamatlıoğlu,Bekir Baran Sıtkı",Cem Karcı,"Çağatay Ulusoy,Salih Bademci,Şifanur Gül",Fırat Yükselir,3.00,23,Onur Güvenatam,"İstanbul,Kırklareli",Veysel Tekşahin,40.00,OGM Pictures,Diğer,Netflix,?,2 Mayıs 2023 - 3 Kasım 2023,2023.00,0,0,0,0,Sona erdi
1241,https://tr.wikipedia.org//wiki/%C5%9Eahmaran_(dizi),Şahmaran,"Aksiyon,Bilim Kurgu,Dram","Şah-ı Mar,–Emine Buzkan Kaynak,[1]",Pınar Bulut,"Umur Turagay,Bertan Başaran","Serenay Sarıkaya,Burak Deniz",Hakan Özer,1.00,8,"Timur Savcı,Burak Sağyaşar","Adana,Tarsus,Datça",Yon Thomas,52.00,Tims&B Productions,Diğer,Netflix,?,20 Ocak 2023 - günümüz,2023.00,0,0,0,0,Sezon arası
1243,https://tr.wikipedia.org//wiki/Yabani,Yabani,Dram,Uyarlama Değil,"Hilal Yıldız,Nazlı Sunlu Kaçan,Serap Gazel,Çisem Katılmaz",Çağatay Tosun,"Halit Özgür Sarı,Simay Barlas,Dolunay Soysert,Bertan Asllani,Yurdaer Okur","Ender Gündüzlü,Metin Arıgül,Can Sanıbelli",1.00,14,"Fatih Aksoy,Mehmet Yiğit Alp",İstanbul,Özkan Bal,130.00,NTC Medya,FOX,?,Stereo,12 Eylül 2023 - günümüz,2023.00,1,1,1,1,Devam ediyor


In [38]:
logging.debug('Adında en fazla kelime bulunan dizi bulundu.')
df[feature_df['Dizi Adındaki Kelime Sayısı'] == feature_df['Dizi Adındaki Kelime Sayısı'].max()]

,Dizi Linki,Dizi Adı,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,Görüntü yönetmeni,Gösterim süresi,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Tarih,Youtube,Facebook,Twitter,Instagram,Durumu
534,https://tr.wikipedia.org//wiki/Esir_%C5%9Eehrin_G%C3%B6zya%C5%9Flar%C4%B1_-_Bir_Ferhat_ile_%C5%9Eirin_Hikayesi,Esir Şehrin Gözyaşları - Bir Ferhat ile Şirin Hikayesi,Dram,Uyarlama Değil,Sevgi Yılmaz,Çağatay Tosun,"Yiğit Özşener,Sedef Avcı",?,1.00,5,Ata Türkoğlu,İstanbul,?,90.00,Koliba Film,FOX,?,Stereo,28 Eylül 2012 - 31 Ekim 2012,2012.00,0,0,0,0,Sona erdi


# Dizi Adında Bağlaç Var Mı ?

In [39]:
def find_baglac(text):
    text = word_tokenize(text)
    for t in text:
        if t in stopwords.words('turkish'):
            return 1

    return 0

In [40]:
feature_df['Dizi Adında Bağlaç'] = df['Dizi Adı'].apply(find_baglac)
logging.debug('Dizi adındaki bağlaç sayısı bulundu.')

In [41]:
logging.debug('Dizi adında bağlaç olan diziler bulundu.')
df[feature_df['Dizi Adında Bağlaç'] == 1].sample(5)

,Dizi Linki,Dizi Adı,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,Görüntü yönetmeni,Gösterim süresi,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Tarih,Youtube,Facebook,Twitter,Instagram,Durumu
555,?,Babalar ve Evlatları,Dram,Uyarlama Değil,Mahsun Kırmızıgül,Cemal Şan,"Deniz Baysal,Murat Ünalmış,Fatma Girik,Serdar Gökhan,Ceren Hindistan",?,1.00,4,Murat Tokat,İstanbul,?,105.00,?,Diğer,?,?,18 Eylül 2012,2012.00,0,0,0,0,Sona erdi
175,https://tr.wikipedia.org//w/index.php?title=Tar%C4%B1k_ve_Di%C4%9Ferleri&action=edit&redlink=1,Tarık ve Diğerleri,"Aile,Aksiyon,Komedi",Uyarlama Değil,"Mahmut Bengi,Oktay Berber",Gürsel Ateş,"Mehmet Püsküllü,Altay İzbırak,İsmet Numanoğlu",?,1.00,30,Salih Asan,İstanbul,Turgay Necipoğlu,90.00,Ser Film,Samanyolu TV,?,?,8 Ekim 2006,2006.00,0,0,0,0,Sona erdi
440,https://tr.wikipedia.org//wiki/A%C5%9Fk_ve_Ceza,Aşk ve Ceza,"Dram,Aşk,Romantik",Uyarlama Değil,Yıldız Tunç,Kudret Sabancı,"Murat Yıldırım,Nurgül Yeşilçay,Feride Çetin,Tomris İncer,Caner Kurtaran",?,2.00,62,Erol Avcı,İstanbul,Mete Şener,120.00,TMC Film,atv,?,Stereo,5 Ocak 2010 - 27 Haziran 2011,2010.00,0,0,0,0,Sona erdi
735,https://tr.wikipedia.org//wiki/Analar_ve_Anneler,Analar ve Anneler,Dram,Uyarlama Değil,Berkun Oya,Mehmet Ada Öztekin,"Hazar Ergüçlü,Binnur Kaya,Okan Yalabık,Sinem Kobal,Nazan Kesal,Ulaş Tuna Astepe,Metin Akdülger",Toygar Işıklı,1.00,9,Kerem Çatay,İstanbul,?,90.00,Ay Yapım,atv,?,Stereo,"22 Ekim 2015,(,2015-10-22,),-,24 Aralık 2015,(,2015-12-24,)",2015.00,0,0,0,0,Yayından kaldırıldı
994,https://tr.wikipedia.org//wiki/Maria_ile_Mustafa,Maria ile Mustafa,"Romantik,Dram",Uyarlama Değil,Birol Elginöz,Faruk Teber,"Jessica May,Hilmi Cem İntepe,Tamer Levent",?,1.00,17,Mehmet Yiğit Alp,"Ürgüp,,,Nevşehir",?,130.00,NTC Medya,atv,?,Stereo,6 Eylül 2020 - 27 Aralık 2020,2020.00,0,1,1,1,Sona erdi


# Dizi Adında Yer İsmi Var Mı ? 

In [42]:
def find_place(text):
    if text != '?':
        doc = nlp(text)
        found = 0
    
        for ent in doc.ents:
            if ent.label_ == 'GPE' or ent.label_ == 'GEO':
                found = 1
    
        return found
    else:
        return -1

In [43]:
nlp = spacy.load('tr_core_news_lg')

In [44]:
feature_df['Dizi Adında Yer İsmi'] = df['Dizi Adı'].apply(find_place)
logging.debug('Dizi adında yer ismi özelliği bulundu.')

In [45]:
logging.debug('Dizi adında yer ismi olan diziler bulundu.')
df[feature_df['Dizi Adında Yer İsmi'] == 1].sample(5)

,Dizi Linki,Dizi Adı,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,Görüntü yönetmeni,Gösterim süresi,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Tarih,Youtube,Facebook,Twitter,Instagram,Durumu
514,?,Sudan Sebepler,"Aile,Komedi",Uyarlama Değil,"Eda Tezcan,Ramazan Demir",Özlem Bayşu,"Murat Akkoyunlu,Oya Aydoğan,Buket Dereoğlu,Ümit Yesin,Kayra Şenocak",?,1.00,2,"Murat Kürüz,Oğuz Ercan",İstanbul,?,90.00,?,Diğer,?,?,2011,2011.00,0,0,0,0,Sona erdi
218,https://tr.wikipedia.org//wiki/Hakk%C4%B1n%C4%B1_Helal_Et,Hakkını Helal Et,Dram,Uyarlama Değil,?,"İsmail Güneş,Naci Çelik Berksoy,",Oyuncular,?,1.00,37,?,İstanbul,?,80.00,?,Samanyolu TV,?,Stereo,"20 Ekim 2007,(,2007-10-20,),[2],[3],[4],-,28 Haziran 2008,(,2008-06-28,),[5],[6]",2007.00,0,0,0,0,Sona erdi
1142,https://tr.wikipedia.org//wiki/K%C4%B1z%C4%B1lc%C4%B1k_%C5%9Eerbeti,Kızılcık Şerbeti,Dram,Uyarlama Değil,"Melis Civelek,Zeynep Gür",Hakan Kırvavaç,"Barış Kılıç,Evrim Alasya,Sıla Türkoğlu,Sibel Taşçıoğlu,Müjde Uzman,Settar Tanrıöğen,Doğukan Güngör","Zeynep Alasya ,Caner Özkan ,Ertan Özkan ,Serkan Ölçer ,Ece Ölçer",2.00,42,Faruk Turgut,İstanbul,?,130.00,Gold Film,Show TV,?,Stereo,28 Ekim 2022 - günümüz,2022.00,1,1,1,1,Devam ediyor
1123,https://tr.wikipedia.org//wiki/Zeytin_A%C4%9Fac%C4%B1_(dizi),Zeytin Ağacı,"Romantik,Dram",Uyarlama Değil,Nuran Evren Şit,"Burcu Alptekin,Erdem Tepegöz","Tuba Büyüküstün,Seda Bakan,Boncuk Yılmaz","Aytaç Bayladı,Özgür Buldum",1.00,8,Onur Güvenatam,İstanbul,Gökhan Tiryaki,50.00,OGM Pictures,Diğer,Netflix,?,28 Temmuz 2022,2022.00,0,0,0,0,Sezon arası
490,https://tr.wikipedia.org//wiki/Avrupa_Avrupa,Avrupa Avrupa,Komedi,Uyarlama Değil,"Fethi Kantarcı,Saygın Delibaş,Barkın Şenüren",Cevriye Demir,"Tarık Ünlüoğlu,Selen Seyven,Burak Serdar Şanal,Bahar Süer,Zuhal Topal,Şehsuvar Aktaş,Kadir Çöpdemir,Günay Karacaoğlu,Ahmet Saraçoğlu,Durukan Çelikkaya",?,3.00,91,Banu Akdeniz,"İstanbul,,,Maltepe",?,100.00,Mia Yapım,TRT 1,?,Stereo,25 Eylül 2011 - 27 Kasım 2013,2011.00,0,0,0,0,Sona erdi


# Dizi Adında Özel İsim Var Mı ?

In [46]:
def find_person(text):
    doc = nlp(text)
    found = 0

    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            found = 1

    return found

In [47]:
feature_df['Dizi Adında Özel İsim'] = df['Dizi Adı'].apply(find_person)
logging.debug('Dizi adında özel isim özelliği bulundu.')

In [48]:
logging.debug('Dizi adında özel isim olan diziler bulundu.')
df[feature_df['Dizi Adında Özel İsim'] == 1].sample(5)

,Dizi Linki,Dizi Adı,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,Görüntü yönetmeni,Gösterim süresi,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Tarih,Youtube,Facebook,Twitter,Instagram,Durumu
333,?,Yaban Gülü,"Dram,Romantik",Uyarlama Değil,Nuran Devres,"Filiz Kaynak,Celal Çimen","Chan Okan,Ali Yaylı,Hatice Şendil,İpek Tenolcay,Damla Debre","Yaşar Kurt,Targan Berk Türe,Sıla Gençoğlu,Efe Bahadır",1.00,?,"Hüseyin Apaydın,Yalçın Şen",İstanbul,?,80.00,?,atv,?,?,2008,2008.00,0,0,0,0,Sona erdi
211,?,Fikrimin İnce Gülü,"Aile,Dram,Romantik",Uyarlama Değil,Rüya İşçileri,Merve Girgin,"Yasemin Balık,Ayşe Nil Şamlıoğlu,Aydan Şener,Kenan Işık,Mehmet Esen",?,1.00,?,Sadık Deveci,İstanbul,?,30.00,?,Diğer,?,?,2007,2007.00,0,0,0,0,Yayından kaldırıldı
900,https://tr.wikipedia.org//wiki/Mehmet%C3%A7ik_Kut%27%C3%BCl_Amare,Mehmetçik: Kut'ül-Amare,"Tarihî,Bilim Kurgu,Aksiyon",Uyarlama Değil,Mehmet Bozdağ,"Mustafa Şevki Doğan,Kamil Aydın","Kaan Taşaner,İsmail Hacıoğlu,İsmail Ege Şaşmaz,Cansu Tosun,Özgü Kaya,İlker Aksum,Serdar Gökhan","Alpay Göltekin,Zeynep Alasya",2.00,33,Mehmet Bozdağ,İstanbul,Efe Kubilay,120.00,Bozdağ Film,TRT 1,?,Stereo,18 Ocak 2018 - 31 Ocak 2019,2018.00,0,0,0,0,Sona erdi
918,https://tr.wikipedia.org//wiki/Hakan:_Muhaf%C4%B1z,Hakan: Muhafız,"Aksiyon,Bilim Kurgu,Dram","Karakalem ve Bir Delikanlının Tuhaf Hikayesi,–Nilüfer İpek Gökdel","Jason George,Yasemin Yılmaz,Emre Özpirinçci,Kerim Ceylan,Atilla Ünsal,Aşkım Özbek","Umut Aral,Gönenç Uyanık,Burcu Alptekin,Can Evrenol,Gökhan Tiryaki","Çağatay Ulusoy,Ayça Ayşin Turan,Hazar Ergüçlü,Okan Yalabık,Burçin Terzioğlu,Yurdaer Okur,Engin Öztürk,Taner Ölmez,Funda Eryiğit",?,4.00,32,Alex Sutherland,"İstanbul,,,Türkiye",Gökhan Tiryaki,45.00,"Netflix,O3 Medya",Diğer,Netflix,?,14 Aralık 2018 - 9 Temmuz 2020,2018.00,0,0,0,0,Sona erdi
200,https://tr.wikipedia.org//wiki/Pertev_Bey%27in_%C3%9C%C3%A7_K%C4%B1z%C4%B1,Pertev Bey'in Üç Kızı,Komedi,Uyarlama Değil,Zülküf Yücel,Tunç Başaran,"Serdar Gökhan,Deniz Evrenol,Deniz Gökçer,Başak Daşman,Damla Sönmez,Orhan Aydın,Itır Esen",?,1.00,6,?,İstanbul,?,80.00,TRT,TRT 1,?,Stereo,2006 - 2006,2006.00,0,0,0,0,Sona erdi


# Türkiye'de Mi Çekildi ?

In [49]:
def in_turkey(text):
    places = text.split(',')
    for place in places:
        if len(geo_df[geo_df['Yer'] == place]) > 0:
            enlem = geo_df.loc[geo_df[geo_df['Yer'] == place].index, 'Enlem'].values[0]
            boylam = geo_df.loc[geo_df[geo_df['Yer'] == place].index, 'Boylam'].values[0]
            if 36 <= enlem <= 42 and 26 <= boylam <= 45:
                return 1
            else:
                return 0
        else:
            return 0

In [50]:
geo_df = pd.read_csv('data/geo_df.csv', index_col=0)
logging.debug('Jeolojik veri seti yüklendi.')
geo_df.head()

,Yer,Enlem,Boylam
0,İstanbul,41.08,29.05
1,İzmir,38.42,27.14
2,İzmit,40.77,29.95
3,Kocaeli,40.77,29.94
4,Türkiye,38.96,34.92


In [51]:
feature_df['in_turkey'] = df['Mekân'].apply(in_turkey)
logging.debug('Türkiye geçiyor özelliği bulundu.')

In [52]:
logging.debug('Türkiyede geçen diziler bulundu.')
df[feature_df['in_turkey'] == 1].sample(5)

,Dizi Linki,Dizi Adı,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,Görüntü yönetmeni,Gösterim süresi,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Tarih,Youtube,Facebook,Twitter,Instagram,Durumu
490,https://tr.wikipedia.org//wiki/Avrupa_Avrupa,Avrupa Avrupa,Komedi,Uyarlama Değil,"Fethi Kantarcı,Saygın Delibaş,Barkın Şenüren",Cevriye Demir,"Tarık Ünlüoğlu,Selen Seyven,Burak Serdar Şanal,Bahar Süer,Zuhal Topal,Şehsuvar Aktaş,Kadir Çöpdemir,Günay Karacaoğlu,Ahmet Saraçoğlu,Durukan Çelikkaya",?,3.00,91,Banu Akdeniz,"İstanbul,,,Maltepe",?,100.00,Mia Yapım,TRT 1,?,Stereo,25 Eylül 2011 - 27 Kasım 2013,2011.00,0,0,0,0,Sona erdi
187,https://tr.wikipedia.org//wiki/Selena_(dizi),Selena,"Bilim Kurgu,Komedi",Uyarlama Değil,"Gani Müjde,Fatih Solmaz,Emre Bülbül,Nilüfer Yenidoğan,Gökhan Karagülle",Bora Onur,"Sinem Kobal,Gökhan Keser,Cansu Demirci,Gizem Güven,Dilara Kurtulmuş,Sinan Çalışkanoğlu,Zuhal Topal",?,3.00,104,"Ali Gündoğdu,İnci Kırhan Gündoğdu",İstanbul,?,96.00,Süreç Film,atv,?,Stereo,10 Eylül 2006 - 18 Ağustos 2009,2006.00,0,0,0,0,Sona erdi
461,?,Kirli Beyaz,Dram,Uyarlama Değil,?,Faruk Teber,"Yıldırım Memişoğlu,Sarp Levendoğlu,Ragıp Savaş,İdil Fırat,Mustafa Uğurlu",?,1.00,6,?,İstanbul,?,120.00,?,FOX,?,?,2010,2010.00,0,0,0,0,Yayından kaldırıldı
426,https://tr.wikipedia.org//wiki/Kader_%C3%87izgisi,Kader Çizgisi,Dram,Uyarlama Değil,Mehmet Uyar,?,"Yunus Günçe,Özer Tunca,Uğur İzgi,Simge Geren,Uğur Taşdemir",?,1.00,15,Mustafa Kartal,İstanbul,?,100.00,Ser Film,Samanyolu TV,?,Stereo,10 Mart 2010 - 16 Haziran 2010,2010.00,0,0,0,0,Sona erdi
0,https://tr.wikipedia.org//wiki/Kaynanalar,Kaynanalar,Komedi,Uyarlama Değil,?,"Tekin Akmansoy,Arzu Akmansoy","Tekin Akmansoy,Leman Çıdamlı,Defne Yalnız,Sevda Aydan,Ege Aydan,Münir Caner,Sermin Hürmeriç,Sezai Aydın",?,14.00,314,?,İstanbul,?,58.00,?,"TRT 1,Kanal D",?,?,19 Mayıs 1974 - 24 Ocak 2005,1974.00,0,0,0,0,Sona erdi


# Yurtdışında Mı Çekildi ?

In [53]:
def out_turkey(text):
    places = text.split(',')
    for place in places:
        if len(geo_df[geo_df['Yer'] == place]) > 0:
            enlem = geo_df.loc[geo_df[geo_df['Yer'] == place].index, 'Enlem'].values[0]
            boylam = geo_df.loc[geo_df[geo_df['Yer'] == place].index, 'Boylam'].values[0]
            if 36 <= enlem <= 42 and 26 <= boylam <= 45:
                return 0
            else:
                return 1
        else:
            return 0

In [54]:
feature_df['out_turkey'] = df['Mekân'].apply(out_turkey)
logging.debug('Yurtdışında geçiyor özelliği bulundu.')

In [55]:
feature_df['in_turkey'].value_counts()

in_turkey
1    1232
0      20
Name: count, dtype: int64

In [56]:
logging.debug('Yurtdışında geçen diziler bulundu.')
df[feature_df['out_turkey'] == 1].sample(5)

,Dizi Linki,Dizi Adı,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,Görüntü yönetmeni,Gösterim süresi,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Tarih,Youtube,Facebook,Twitter,Instagram,Durumu
205,https://tr.wikipedia.org//wiki/Elveda_Rumeli,Elveda Rumeli,"Dram,Tarihî",Uyarlama Değil,"Alican Yaraş,Nuran Evren Şit,Özge Efendioğlu,Murat Levent,Ertan Saban","Serdar Akar,Doğan Ümit Karaca,Özer Kızıltan","Erdal Özyağcılar,Tolgahan Sayışman,Şebnem Sönmez,Gökhan Mete,Mustafa Jasar,Filiz Ahmet,Erman Saban,Berrak Tüzünataç,Suzan Akbelge,Luran Ahmeti,Ertan Saban,Bedia Begovska,Tuna Orhan,Perihan Tuna,Petar Mircevski,Selahattin Bilal,Elyesa Kaso",?,3.00,83,Tarkan Karlıdağ,"Manastır,,,Makedonya",?,90.00,Adam Film,atv,?,Stereo,20 Eylül 2007 - 19 Ekim 2009,2007.00,0,0,0,0,Sona erdi
713,https://tr.wikipedia.org//wiki/Buyur_Bi%27De_Burdan_Bak,Buyur Bi'De Burdan Bak,Komedi,Uyarlama Değil,"Umut Kurt,Melikşah Altuntaş",Ebru Yalçın,"Giray Altınok,Doğan Akdoğan,Özgün Bayraktar,Alper Baytekin,Bilal Çatalçekiç,Dilşah Demir,Aydın Sezgin,Kerem Özdoğan",?,2.00,24,Necati Akpınar,BKM,?,90.00,BKM Film,"Kanal D,TV8",?,Stereo,29 Temmuz 2015 - 5 Temmuz 2018,2015.00,0,0,0,0,Sona erdi
1164,https://tr.wikipedia.org//wiki/Pera_Palas%27ta_Gece_Yar%C4%B1s%C4%B1,Pera Palas'ta Gece Yarısı,"Dram,Tarihî",Midnight at the Pera Palace - Charles King,Elif Usman,"Emre Şahin,Nisan Dağ","Hazal Kaya,Tansu Biçer,Selahattin Paşalı",?,1.00,8,?,"Pera Palas,,,İstanbul",Clint Lealos,45.00,Karga Seven Pictures,Diğer,Netflix,?,3 Mart 2022 - günümüz,2022.00,0,0,0,0,Sezon arası
594,https://tr.wikipedia.org//wiki/Arkada%C5%9F%C4%B1m_Ho%C5%9Fgeldin,Arkadaşım Hoşgeldin,Komedi,Uyarlama Değil,Fırat Doğu Parlak,Ebru Yalçın,"Tolga Çevik,Ezgi Mola,Özer Atik,Sarp Bozkurt,Fırat Doğu Parlak,",Özer Atik,2.00,39,Necati Akpınar,"BKM,(BKM)",?,90.00,BKM,Kanal D,?,Stereo,31 Aralık 2013 - 31 Ocak 2015,2013.00,0,0,0,0,Sona erdi
682,https://tr.wikipedia.org//wiki/Dirili%C5%9F_Ertu%C4%9Frul,Diriliş Ertuğrul,"Tarihî,Bilim Kurgu,Aksiyon,Dram",Uyarlama Değil,"Mehmet Bozdağ,Atilla Engin,Ozan Bodur,Aslı Zeynep Peker Bozdağ,Serdar Özönalan",Metin Günay,"Engin Altan Düzyatan,Kaan Taşaner,Hülya Darcan,Esra Bilgiç,Didem Balçın,Serdar Gökhan,Hakan Vanlı,Serdar Deniz,Levent Öktem,Hande Subaşı,Burcu Kıratlı,Barış Bağcı,Uğur Güneş,Ayberk Pekcan,Osman Soykut,Mehmet Çevik,Sezgin Erdemir,Kürşat Alnıaçık,Cem Uçan,Erden Alkan,Gülsim Ali,Gülçin Santırcıoğlu,Murat Garipağaoğlu,Çağdaş Onur Öztürk,Cengiz Coşkun,Cavit Çetin Güner,Nurettin Sönmez,Cemal Hünal,Burak Hakkı,Batuhan Karacakaya,Hande Soral,İlker Aksum,Ali Ersan Duru",Zeynep Alasya,5.00,150,Mehmet Bozdağ,"Riva,,,Beykoz,,,İstanbul,[1]",?,120.00,Tekden Film,TRT 1,"Netflix,Youtube",Stereo,10 Aralık 2014 - 29 Mayıs 2019,2014.00,0,0,0,0,Sona erdi


# Başrol Sayısı

In [57]:
def act_count(text):
    if text != '?':
        text_arr = []
        texts = text.split(',')
        for t in texts:
            if t not in ['?', '', ' ']:
                if t not in stopwords.words('turkish'):
                    text_arr.append(t.strip())
                    
        return len(text_arr)
    return -1

In [58]:
feature_df['act_count'] = df['Başrol'].apply(act_count)
logging.debug('Toplam başrol sayısı özelliği bulundu.')

In [59]:
logging.debug('En fazla başrole sahip dizi bulundu.')
df[feature_df['act_count'] == feature_df['act_count'].max()]

,Dizi Linki,Dizi Adı,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,Görüntü yönetmeni,Gösterim süresi,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Tarih,Youtube,Facebook,Twitter,Instagram,Durumu
723,https://tr.wikipedia.org//wiki/E%C5%9Fk%C4%B1ya_D%C3%BCnyaya_H%C3%BCk%C3%BCmdar_Olmaz_(dizi),Eşkıya Dünyaya Hükümdar Olmaz,"Aksiyon,Polisiye,Dram",Uyarlama Değil,"Raci Şaşmaz,Bahadır Özdener,Berna Aruz,","Onur Tan,Zübeyr Şaşmaz,Baran Özçaylan,",",Oktay Kaynarca,Deniz Çakır,Müjde Uzman,Yunus Emre Yıldırımer,Ozan Akbaba,Tarık Ünlüoğlu,Oktay Kaynarca,Deniz Çakır,Sanem Çelik,Yunus Emre Yıldırımer,Ozan Akbaba,Tarık Ünlüoğlu,3. sezon,Oktay Kaynarca,Deniz Çakır,Sanem Çelik,Mesut Akusta,Olgun Şimşek,Yunus Emre Yıldırımer,Ozan Akbaba,Tarık Ünlüoğlu,4. sezon,Oktay Kaynarca,Mustafa Üstündağ,Hüseyin Avni Danyal,Yunus Emre Yıldırımer,Ozan Akbaba,Tarık Ünlüoğlu,5. sezon,Oktay Kaynarca,Naz Elmas,Mustafa Üstündağ,Hüseyin Avni Danyal,Yunus Emre Yıldırımer,Ozan Akbaba,Burak Sergen,6. sezon,Oktay Kaynarca,Sanem Çelik,Hüseyin Avni Danyal,Yunus Emre Yıldırımer,Ozan Akbaba,Ali Sürmeli","Ayşe Önder,Levent Güneş",6.00,199,"Raci Şaşmaz,atv İç Yapımlar,",İstanbul,Veysel Kılınç,120.00,?,atv,?,Stereo,8 Eylül 2015 - 15 Haziran 2021,2015.00,1,0,1,1,Yayından kaldırıldı


In [60]:
logging.debug('En az başrole sahip dizi bulundu.')
df[feature_df['act_count'] == feature_df['act_count'].min()]

,Dizi Linki,Dizi Adı,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,Görüntü yönetmeni,Gösterim süresi,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Tarih,Youtube,Facebook,Twitter,Instagram,Durumu
691,https://tr.wikipedia.org//wiki/Benim_Ad%C4%B1m_G%C3%BCltepe,Benim Adım Gültepe,Dram,Uyarlama Değil,Vural Yaşaroğlu,"Zeynep Günay,Deniz Koloş",?,Uğur Akyürek,1.00,8,E. Ayşe Durmaz,"İzmir,İstanbul",Sedat Yücel,100.00,D Productions,Kanal D,?,Stereo,3 Eylül 2014 - 3 Kasım 2014,2014.00,0,0,0,0,Yayından kaldırıldı
750,?,Son Çıkış,"Dram,Aile,Polisiye",Uyarlama Değil,?,?,?,?,1.00,17,?,İstanbul,?,120.00,Estet Film,Diğer,?,?,1 Haziran 2015,2015.00,0,0,0,0,Sona erdi


# Uyarlama mı ?

In [61]:
feature_df['book_to_movie'] = df['Uyarlama'].apply(lambda x: 1 if x != 'Uyarlama Değil' else 0)
logging.debug('Uyarlama özelliği bulundu.')

In [62]:
feature_df['book_to_movie'].value_counts()

book_to_movie
0    1167
1      85
Name: count, dtype: int64

In [63]:
logging.debug('Uyarlama olan diziler bulundu.')
df[feature_df['book_to_movie'] == 1].sample(5)

,Dizi Linki,Dizi Adı,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,Görüntü yönetmeni,Gösterim süresi,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Tarih,Youtube,Facebook,Twitter,Instagram,Durumu
959,https://tr.wikipedia.org//wiki/A%C5%9Fk_A%C4%9Flat%C4%B1r,Aşk Ağlatır,Dram,Love That Makes You Cry,"Yelda Eroğlu,Yeşim Çıtak",Gökçen Usta,"Deniz Can Aktaş,Hafsanur Sancaktutan,Yağız Can Konyalı,Eda Şölenci",?,1.00,16,Faruk Bayhan,İstanbul,?,120.00,MF Yapım,Show TV,?,Stereo,8 Eylül 2019 - 22 Aralık 2019,2019.00,0,0,0,1,Sona erdi
650,https://tr.wikipedia.org//wiki/Not_Defteri_(dizi),Not Defteri,Gençlik,Ölü Ozanlar Derneği,"Alphan Dikmen,Başak Angigün",Ilgaz Giritlioğlu,"Fatih Al,Ebru Özkan,Ekin Mert Daymaz,Tarık Özcan Ündüz,Beste Kökdemir",Harems,1.00,13,Şükrü Avşar,İstanbul,?,100.00,Avşar Film,FOX,?,Stereo,"26 Mart 2014,(,2014-03-26,),-,25 Haziran 2014,(,2014-06-25,)",2014.00,0,0,0,0,"Yayından kaldırıldı,[1]"
376,https://tr.wikipedia.org//wiki/Ezel_(dizi),Ezel,"Aksiyon,Dram","Monte Kristo Kontu,–,Alexandre Dumas","Kerem Deren,Pınar Bulut",Uluç Bayraktar,"Kenan İmirzalıoğlu,Cansu Dere,Yiğit Özşener,Barış Falay,Tuncel Kurtiz,Haluk Bilginer",Toygar Işıklı,2.00,71,Kerem Çatay,"İstanbul,Ankara,Kıbrıs",?,90.00,Ay Yapım,"Show TV,atv",?,Stereo,28 Eylül 2009 - 20 Haziran 2011,2009.00,0,0,0,0,Sona erdi
1162,https://tr.wikipedia.org//wiki/Hakim_(dizi),Hakim,"Dram,Polisiye,Aksiyon",Your Honor,"Yılmaz Şahin,Çağla Kızılelma ,Gökhan Şeker ,Ali Doğançay","Erkan Tunç ,Süleyman Mert Özdemir ,Osman Taşcı,Günay Günaydın,","Erdal Beşikçioğlu,Ebru Özkan,Uğur Yücel,Yurdaer Okur",?,1.00,10,Hayri Aslan,İstanbul,?,120.00,Limon Film,atv,?,Stereo,4 Nisan 2022 - 13 Haziran 2022,2022.00,0,0,1,1,Sona erdi
1140,https://tr.wikipedia.org//wiki/Yakamoz_S-245,Yakamoz S-245,"Aksiyon,Bilim Kurgu,Dram",The Old Axolotl,"Jason George,Atasay Koç,Cansu Çoban,Sami Berat Marçalı,Murat Uyurkulak","Tolga Karaçelik,Umut Aral","Kıvanç Tatlıtuğ,Özge Özpirinçci,Ertan Saban",Ahmet Bilgiç,1.00,7,Onur Güvenatam,"Foça,,,Ayvalık",?,45.00,OGM Pictures,Diğer,Netflix,?,20 Nisan 2022,2022.00,0,0,0,0,Yayımlandı


# Sosyal Medya

In [64]:
feature_df['on_youtube'] = df['Youtube']
logging.debug('Youtube hesabı özelliği bulundu.')
feature_df['on_facebook'] = df['Facebook']
logging.debug('Facebook hesabı özelliği bulundu.')
feature_df['on_twitter'] = df['Twitter']
logging.debug('Twitter hesabı özelliği bulundu.')
feature_df['on_instagram'] = df['Instagram']
logging.debug('Instagram hesabı özelliği bulundu.')
feature_df['total_sm_accounts'] = feature_df['on_youtube'] + feature_df['on_facebook'] + feature_df['on_twitter'] + feature_df['on_instagram']
logging.debug('Toplam sosyal medya hesabı sayısı özelliği bulundu.')

In [65]:
feature_df['on_youtube'].value_counts()

on_youtube
0    1125
1     127
Name: count, dtype: int64

In [66]:
feature_df['on_facebook'].value_counts()

on_facebook
0    1152
1     100
Name: count, dtype: int64

In [67]:
feature_df['on_twitter'].value_counts()

on_twitter
0    1040
1     212
Name: count, dtype: int64

In [68]:
feature_df['on_instagram'].value_counts()

on_instagram
0    1064
1     188
Name: count, dtype: int64

In [69]:
feature_df['total_sm_accounts'].value_counts()

total_sm_accounts
0    991
1     74
2     70
4     62
3     55
Name: count, dtype: int64

# Yayın Günü

In [70]:
def remove_parentheses(text):
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'\[[^]]*\]', '', text)
    text = re.sub('1. sezon', '', text)
    text = re.sub('2. sezon', '', text)
    #text = re.sub(',', '', text)
    text = text.strip()
    return text

In [71]:
def find_weekday(text):
    text = remove_parentheses(text)
    if text != '?':
        text = text.replace(',', '')
        if text.startswith(':'):
            text = text.replace(':', '')
            text = text.strip()
        text = text.split('-')[0]
        text = text.strip()
        if len(text.split(' ')) >= 3:
            text = text.split(' ')[:3]
        else:
            return '?'
        text = ' '.join([t for t in text])
        text = text.replace('Ocak', 'January')
        text = text.replace('Şubat', 'February')
        text = text.replace('Mart', 'March')
        text = text.replace('Nisan', 'April')
        text = text.replace('Mayıs', 'May')
        text = text.replace('Haziran', 'June')
        text = text.replace('Temmuz', 'July')
        text = text.replace('Ağustos', 'August')
        text = text.replace('Eylül', 'September')
        text = text.replace('Ekim', 'October')
        text = text.replace('Kasım', 'November')
        text = text.replace('Aralık', 'December')
        #print(text)

        datetime_obj = datetime.strptime(text, '%d %B %Y')
        return datetime_obj.strftime('%A')
        
    else:
        return text

In [72]:
feature_df['Publication Day'] = df['Yayın tarihi'].apply(find_weekday)
logging.debug('Dizinin yayınlandığı gün özelliği bulundu.')

In [73]:
feature_df.isnull().sum()

Tarih                          0
Dizi Adı                       0
Dizi Adı Uzunluğu              0
Dizi Adındaki Kelime Sayısı    0
Dizi Adında Bağlaç             0
Dizi Adında Yer İsmi           0
Dizi Adında Özel İsim          0
in_turkey                      0
out_turkey                     0
act_count                      0
book_to_movie                  0
on_youtube                     0
on_facebook                    0
on_twitter                     0
on_instagram                   0
total_sm_accounts              0
Publication Day                0
dtype: int64

In [74]:
feature_df['Tarih'].unique()

array([1974., 1975., 1983., 1984., 1985., 1986., 1987., 1988., 1989.,
       1990., 1991., 1992., 1993., 1994., 1995., 1996., 1997., 1998.,
       1999., 2000., 2001., 2002., 2003., 2004., 2005., 2006., 2007.,
       2008., 2009., 2010., 2011., 2012., 2013., 2014., 2015., 2016.,
       2017., 2018., 2019., 2020., 2021., 2022., 2023.])

In [75]:
feature_df['Publication Day'].fillna(method='bfill', inplace=True)

/tmp/ipykernel_2956/4163069526.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feature_df['Publication Day'].fillna(method='bfill', inplace=True)


# Yaz Dizisi Mi ?

In [76]:
def find_summer_series(text):
    text = remove_parentheses(text)
    if text != '?':
        text = text.replace(',', '')
        if text.startswith(':'):
            text = text.replace(':', '')
            text = text.strip()
        text = text.split('-')[0]
        text = text.strip()
        if len(text.split(' ')) >= 3:
            text = text.split(' ')[:3]
        else:
            return -1
        text = ' '.join([t for t in text])
        text = text.replace('Ocak', 'January')
        text = text.replace('Şubat', 'February')
        text = text.replace('Mart', 'March')
        text = text.replace('Nisan', 'April')
        text = text.replace('Mayıs', 'May')
        text = text.replace('Haziran', 'June')
        text = text.replace('Temmuz', 'July')
        text = text.replace('Ağustos', 'August')
        text = text.replace('Eylül', 'September')
        text = text.replace('Ekim', 'October')
        text = text.replace('Kasım', 'November')
        text = text.replace('Aralık', 'December')
        #print(text)

        if 'June' in text or 'July' in text:
            return 1
        else:
            return 0
        
    else:
        return -1

In [77]:
feature_df['summer_series'] = df['Yayın tarihi'].apply(find_summer_series)
logging.debug('Dizinin yayınlandığı gün özelliği bulundu.')

In [78]:
feature_df['summer_series'].value_counts()

summer_series
 0    901
 1    187
-1    164
Name: count, dtype: int64

# Ücretli Kanalda Mı Yayınlanıyor ?

In [79]:
def find_paid_channel(text):
    text = remove_parentheses(text)
    channels = text.split(',')
    for c in channels:
        if len(channel_df[channel_df['Kanal Adı'] == c]) > 0:
            state = channel_df.loc[channel_df[channel_df['Kanal Adı'] == c].index, 'Fiyat'].values[0]
            if state == 'Ücretsiz':
                return 0
            else:
                return 1

        else:
            return -1

In [80]:
channel_df = pd.read_csv('data/channels.csv', index_col=0)

In [81]:
feature_df['paid_channel'] = df['Kanal'].apply(find_paid_channel)
logging.debug('Dizinin yayınlandığı kanal ücreti özelliği bulundu.')

In [82]:
feature_df['paid_channel'].value_counts()

paid_channel
 0    1009
-1     243
Name: count, dtype: int64

# Türler

In [83]:
feature_df['Aile'] = df['Tür'].str.contains('Aile').astype(int)
feature_df['Aksiyon'] = df['Tür'].str.contains('Aksiyon').astype(int)
feature_df['Aşk'] = df['Tür'].str.contains('Aşk').astype(int)
feature_df['Bilim Kurgu'] = df['Tür'].str.contains('Bilim Kurgu').astype(int)
feature_df['Dram'] = df['Tür'].str.contains('Dram').astype(int)
feature_df['Gençlik'] = df['Tür'].str.contains('Gençlik').astype(int)
feature_df['Gerilim'] = df['Tür'].str.contains('Gerilim').astype(int)
feature_df['Komedi'] = df['Tür'].str.contains('Komedi').astype(int)
feature_df['Polisiye'] = df['Tür'].str.contains('Polisiye').astype(int)
feature_df['Romantik'] = df['Tür'].str.contains('Romantik').astype(int)
feature_df['Tarihî'] = df['Tür'].str.contains('Tarihî').astype(int)

In [84]:
logging.debug('Dizinin türü özelliği bulundu.')

In [85]:
feature_df.head()

,Tarih,Dizi Adı,Dizi Adı Uzunluğu,Dizi Adındaki Kelime Sayısı,Dizi Adında Bağlaç,Dizi Adında Yer İsmi,Dizi Adında Özel İsim,in_turkey,out_turkey,act_count,book_to_movie,on_youtube,on_facebook,on_twitter,on_instagram,total_sm_accounts,Publication Day,summer_series,paid_channel,Aile,Aksiyon,Aşk,Bilim Kurgu,Dram,Gençlik,Gerilim,Komedi,Polisiye,Romantik,Tarihî
0,1974.00,Kaynanalar,10,1,0,0,0,1,0,8,0,0,0,0,0,0,Sunday,0,0,0,0,0,0,0,0,0,1,0,0,0
1,1975.00,Aşk-ı Memnu,11,2,0,0,0,1,0,5,0,0,0,0,0,0,Saturday,0,0,1,0,0,0,1,0,0,0,0,0,0
2,1983.00,Kartallar Yüksek Uçar,21,3,0,0,0,1,0,5,0,0,0,0,0,0,Saturday,0,-1,0,0,0,0,0,0,0,0,1,0,0
3,1983.00,Üç İstanbul,11,2,0,0,0,1,0,5,0,0,0,0,0,0,?,-1,0,0,0,0,0,1,0,0,0,0,0,0
4,1984.00,Küçük Ağa,9,2,0,0,0,1,0,4,0,0,0,0,0,0,Sunday,0,0,0,1,0,0,1,0,0,0,0,0,1


# Senaristlerin Görev Aldığı Toplam Dizi Sayısı

In [86]:
senaristler = []

In [87]:
def find_row_count(text, arr):
    if text != '?':
        texts = text.split(',')
        for t in texts:
            if t not in stopwords.words('turkish') and len(t) > 0 and t != '?':
                arr.append(t.strip())
                    
        countered_arr = dict(Counter(arr))
        #print(countered_senarist)
        arr_total = 0
        for t in texts:
            if t not in stopwords.words('turkish') and len(t) > 0 and t != '?':
                try:
                    arr_total += countered_arr[t]
                except:
                    pass
    
        return float(arr_total / len(texts))
    else:
        return -1

In [88]:
feature_df['Senaristlerin Dizi Sayısı'] = df['Senarist'].apply(lambda x: find_row_count(x, senaristler))
logging.debug('Dizinin senaristlerinin görev aldığı ortalama dizi sayısı özelliği bulundu.')

In [89]:
Counter(senaristler)

Counter({'Birol Güven': 25,
         'Gani Müjde': 22,
         'Mahinur Ergun': 22,
         'Meriç Acemi': 18,
         'Caner Güler': 17,
         'Gökhan Horzum': 16,
         'Eylem Canpolat': 16,
         'Makbule Kosif': 15,
         'Sema Ergenekon': 15,
         'Tayfun Güneyer': 14,
         'Melek Gençoğlu': 13,
         'Ece Yörenç': 13,
         'Gül Abus Semerci': 13,
         'Can Sinan': 12,
         'Nuran Evren Şit': 12,
         'Eda Tezcan': 12,
         'Hakan Haksun': 11,
         'Resul Ertaş': 11,
         'Ayça Mutlugil': 11,
         'Deniz Akçay': 11,
         'Baykut Badem': 11,
         'Yelda Eroğlu': 11,
         'Nuran Devres': 10,
         'Neşe Cehiz': 10,
         'Murat Aras': 10,
         'Erkan Birgören': 10,
         'Funda Alp': 10,
         'Ahmet Saatçioğlu': 10,
         'Bahadır Özdener': 9,
         'Sergin Akyaz': 9,
         'Özgür Evren Heptürk': 9,
         'Rüya İşçileri': 9,
         'Barış Erdoğan': 9,
         'Gamze Özer': 9,
      

# Ödül Almış Senarist Sayısı

In [90]:
odullu_senaristler = {}

In [91]:
pantene_senarist_df = pd.read_csv('data/pantene_odullu_senaristler.csv', index_col=0)
pantene_senarist_df.head()

,Yıl,Senarist,Yapım
0,1998,Kandemir Konduk,Mahallenin Muhtarları
1,1999,Kandemir Konduk,Mahallenin Muhtarları
2,2000,Tayfun Güneyer,Yılan Hikayesi
3,2001,Kandemir Konduk,Ana Kuzusu
4,2012,Gülse Birsel,Yalan Dünya


In [92]:
def find_award_winning(text, arr, df_target, target):
    arr_total = 0
    if text[0] != '?':
        if text['Tarih'] != '?':
            texts = text[df_target].split(',')
            for t in texts:
                if t not in stopwords.words('turkish') and len(t) > 0 and t != '?':                
                    temp_df = arr[arr['Yıl'] <= int(text['Tarih'])]
                    if t in temp_df[target].unique():
                        arr_total += 1
    
            return arr_total
        else:
            return -1
    else:
        return -1

In [93]:
feature_df['Ödüllü Senarist Sayısı'] = df[['Tarih', 'Senarist']].apply(lambda row: find_award_winning(row, pantene_senarist_df, 'Senarist', 'Senarist'), axis=1)
logging.debug('Dizinin senaristlerdeki ödüllü senarist sayısı özelliği bulundu.')

/tmp/ipykernel_2956/707262048.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if text[0] != '?':


In [94]:
feature_df['Ödüllü Senarist Sayısı'].value_counts()

Ödüllü Senarist Sayısı
0    1226
1      26
Name: count, dtype: int64

# Senaristlerin Aldığı Toplam Ödül Sayısı

In [95]:
def find_award_winning_count(text, arr, df_target, pantene_arr, target):
    arr_total = 0
    if text[0] != '?':
        if text['Tarih'] != '?':
            texts = text[df_target].split(',')
            for t in texts:
                if t not in stopwords.words('turkish') and len(t) > 0 and t != '?':
                    if t not in arr:
                        arr[t] = 0
                    
                    temp_df = pantene_arr[pantene_arr['Yıl'] <= int(text['Tarih'])]
                    if t in temp_df[target].unique():
                        arr[t] += 1
    
                    arr_total += arr[t]
    
            return float(arr_total / len(texts))
        else:
            return -1
    else:
        return -1

In [96]:
feature_df['Senaristlerin Aldığı Ödül Sayısı'] = df[['Tarih', 'Senarist']].apply(lambda row: find_award_winning_count(row, odullu_senaristler, 'Senarist', pantene_senarist_df, 'Senarist'), axis=1)
logging.debug('Dizinin senaristlerdeki aldığı toplam ortalama ödül sayısı özelliği bulundu.')

/tmp/ipykernel_2956/3226907082.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if text[0] != '?':


# Başrollerin Görev Aldığı Dizi Sayısı

In [97]:
basroller = []

In [98]:
feature_df['Başrollerin Dizi Sayısı'] = df['Başrol'].apply(lambda x: find_row_count(x, basroller))
logging.debug('Dizinin başrollerinin görev aldığı ortalama dizi sayısı özelliği bulundu.')

In [99]:
Counter(basroller)

Counter({'Oktay Kaynarca': 20,
         'Ahmet Mümtaz Taylan': 19,
         'Haluk Bilginer': 18,
         'Fikret Kuşkan': 17,
         'Nurgül Yeşilçay': 17,
         'İsmail Hacıoğlu': 17,
         'Emre Kınay': 17,
         'Dolunay Soysert': 17,
         'Sarp Levendoğlu': 15,
         'Engin Altan Düzyatan': 15,
         'Tamer Karadağlı': 14,
         'Deniz Uğur': 14,
         'Yiğit Özşener': 14,
         'Birce Akalay': 14,
         'Erkan Can': 13,
         'Asuman Dabak': 13,
         'İlker Aksum': 13,
         'Burak Hakkı': 13,
         'Zafer Algöz': 13,
         'Melisa Sözen': 13,
         'Berk Oktay': 13,
         'Tülin Özen': 13,
         'Erdal Beşikçioğlu': 13,
         'Çetin Tekindor': 12,
         'Altan Erkekli': 12,
         'Yurdaer Okur': 12,
         'Mustafa Üstündağ': 12,
         'Mesut Akusta': 12,
         'Vildan Atasever': 12,
         'Pelin Akil': 12,
         'Şebnem Bozoklu': 12,
         'Hande Soral': 12,
         'Seçkin Özdemir': 12,
     

# Ödül Almış Başrol Sayısı

In [100]:
odullu_basroller = {}

In [101]:
pantene_oyuncu_df = pd.read_csv('data/pantene_odullu_oyuncular.csv', index_col=0)
pantene_oyuncu_df.head()

,Yıl,Oyuncu,Yapım
0,1994,Erdal Özyağcılar,Bizimkiler
1,1994,Yasemin Yalçın,Haşlama Taşlama
2,1994,Yasemin Yalçın,Gülelim Eğlenelim
3,1994,Yasemin Yalçın,Bilinmiyor
4,1994,Pınar Deniz,Yargı


In [102]:
feature_df['Ödüllü Başrol Sayısı'] = df[['Tarih', 'Başrol']].apply(lambda row: find_award_winning(row, pantene_oyuncu_df, 'Başrol', 'Oyuncu'), axis=1)
logging.debug('Dizinin başrollerindeki ödüllü başrol sayısı özelliği bulundu.')

/tmp/ipykernel_2956/707262048.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if text[0] != '?':


In [103]:
feature_df['Ödüllü Başrol Sayısı'].value_counts()

Ödüllü Başrol Sayısı
0    903
1    280
2     61
3      7
4      1
Name: count, dtype: int64

# Başrollerin Aldığı Toplam Ödül Sayısı

In [104]:
feature_df['Başrollerin Aldığı Ödül Sayısı'] = df[['Tarih', 'Başrol']].apply(lambda row: find_award_winning_count(row, odullu_basroller, 'Başrol', pantene_oyuncu_df, 'Oyuncu'), axis=1)
logging.debug('Dizinin başrollerinin aldığı toplam ortalama ödül sayısı özelliği bulundu.')

/tmp/ipykernel_2956/3226907082.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if text[0] != '?':


# Yönetmenlerin Görev Aldığı Dizi Sayısı

In [105]:
yonetmenler = []

In [106]:
feature_df['Yönetmenlerin Dizi Sayısı'] = df['Yönetmen'].apply(lambda x: find_row_count(x, yonetmenler))
logging.debug('Dizinin yönetmenlerinin görev aldığı ortalama dizi sayısı özelliği bulundu.')

In [107]:
Counter(yonetmenler)

Counter({'Metin Balekoğlu': 20,
         'Şenol Sönmez': 18,
         'Sadullah Celen': 17,
         'Mustafa Şevki Doğan': 17,
         'Volkan Kocatürk': 17,
         'Ersoy Güler': 16,
         'Çağatay Tosun': 16,
         'Cevdet Mercan': 16,
         'Serdar Akar': 15,
         'Faruk Teber': 15,
         'Sadullah Şentürk': 14,
         'Adnan Güler': 14,
         'Onur Tan': 14,
         'Barış Yöş': 14,
         'Taner Akvardar': 13,
         'Ömer Uğur': 13,
         'Gürsel Ateş': 13,
         'Hakan Algül': 13,
         'Cemal Şan': 13,
         'Feride Kaytan': 13,
         'Doğan Ümit Karaca': 13,
         'Osman Taşcı': 13,
         'Bülent İşbilen': 13,
         'Emre Kabakuşak': 13,
         'Altan Dönmez': 13,
         'Aydın Bulut': 12,
         'Bora Tekay': 12,
         'Özer Kızıltan': 12,
         'Kartal Çidamlı': 12,
         'Bora Onur': 12,
         'Ali Bilgin': 12,
         'Murat Onbul': 12,
         'Osman Sınav': 11,
         'Hilal Saral': 11,
         

# Ödül Almış Yönetmen Sayısı

In [108]:
odullu_yonetmenler = {}

In [109]:
pantene_yonetmen_df = pd.read_csv('data/pantene_odullu_yonetmenler.csv', index_col=0)
pantene_yonetmen_df.head()

,Yıl,Yönetmen,Yapım
0,1999,Osman Sınav,Deli Yürek
1,2000,Nihat Durak,Yılan Hikayesi
2,2001,Osman Sınav,Deli Yürek
3,2012,Çağatay Tosun,Suskunlar
4,2013,Uluç Bayraktar ve Cem Karcı,Karadayı


In [110]:
feature_df['Ödüllü Yönetmen Sayısı'] = df[['Tarih', 'Yönetmen']].apply(lambda row: find_award_winning(row, pantene_yonetmen_df, 'Yönetmen', 'Yönetmen'), axis=1)
logging.debug('Dizinin yönetmen ödüllü yönetmen sayısı özelliği bulundu.')

/tmp/ipykernel_2956/707262048.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if text[0] != '?':


In [111]:
feature_df['Ödüllü Yönetmen Sayısı'].value_counts()

Ödüllü Yönetmen Sayısı
0    1198
1      50
2       3
3       1
Name: count, dtype: int64

# Yönetmenlerin Aldığı Toplam Ödül Sayısı

In [112]:
feature_df['Yönetmenlerin Aldığı Ödül Sayısı'] = df[['Tarih', 'Yönetmen']].apply(lambda row: find_award_winning_count(row, odullu_yonetmenler, 'Yönetmen', pantene_yonetmen_df, 'Yönetmen'), axis=1)
logging.debug('Dizinin yönetmenlerinin aldığı toplam ortalama ödül sayısı özelliği bulundu.')

/tmp/ipykernel_2956/3226907082.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if text[0] != '?':


# Görüntü Yönetmenlerinin Dizi Sayısı

In [143]:
goruntu_yonetmenleri = []

In [145]:
df.sample()

,Dizi Linki,Dizi Adı,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,Görüntü yönetmeni,Gösterim süresi,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Tarih,Youtube,Facebook,Twitter,Instagram,Durumu
225,https://tr.wikipedia.org//wiki/Annem,Annem,Dram,Uyarlama Değil,Gamze Özer,Faruk Teber,"Vahide Perçin,Duygu Yetiş,Talat Bulut,İlhan Şeşen,Emel Göksu",Kıraç,2.00,64,Faruk Turgut,İstanbul,?,100.00,Gold Film,Kanal D,?,Stereo,6 Eylül 2007 - 28 Mayıs 2009,2007.00,0,0,0,0,Sona erdi


In [146]:
feature_df['Görüntü Yönetmenlerinin Dizi Sayısı'] = df['Görüntü yönetmeni'].apply(lambda x: find_row_count(x, goruntu_yonetmenleri))

In [147]:
Counter(goruntu_yonetmenleri)

Counter({'Veysel Tekşahin': 9,
         'Taylan Sancaktar': 8,
         'Kemal Şanlı': 7,
         'Mustafa Nuri Eser': 7,
         'Serdar Armutlu': 7,
         'Burak Demirdelen': 7,
         'Eyüp Boz': 7,
         'Erhan Ergün': 7,
         'Sedat Yücel': 6,
         'Selahattin Sancaklı': 6,
         'Hüseyin Tunç': 6,
         'Tevfik Şenol': 5,
         'Altuğ Kasapbaşoğlu': 5,
         'Aras Demiray': 5,
         'Baybars Tekin': 5,
         'Ahmet Bayer': 5,
         'Erhan Makar': 5,
         'Barış Milan': 5,
         'Özgür Polat': 5,
         'Necmettin Akdeniz': 5,
         'Halil Arif Nurdan': 4,
         'Mete Şener': 4,
         'Ferhan Akgün': 4,
         'Ferhat Öçmen': 4,
         'Yalçın Yadel': 4,
         'Mahir Gül': 4,
         'Yusuf Akkuş': 4,
         'Sinan Onur': 4,
         'Atakan Küçükparlak': 4,
         'Burak Kanbir': 4,
         'Vedat Dikmetaş': 4,
         'Yon Thomas': 4,
         'Ümit Çakmaksoy': 4,
         'Efe Kubilay': 4,
         'Bertan Ö

# Yapım Şirketleri Dizi Sayısı

In [148]:
yapim_sirketleri = []

In [149]:
feature_df['Yapım Şirketleri Dizi Sayısı'] = df['Yapım şirketi'].apply(lambda x: find_row_count(x, yapim_sirketleri))

In [150]:
Counter(yapim_sirketleri)

Counter({'Ay Yapım': 52,
         'Süreç Film': 51,
         'Gold Film': 39,
         'Medyapım': 32,
         'D Productions': 28,
         'O3 Medya': 26,
         'MF Yapım': 25,
         'NTC Medya': 24,
         'Erler Film': 22,
         'TRT': 20,
         'Limon Film': 20,
         'Pastel Film': 20,
         'TMC Film': 19,
         'Avşar Film': 18,
         'Koliba Film': 18,
         'Tims Productions': 17,
         'Sinegraf': 16,
         'Most Production': 15,
         'BKM': 15,
         'OGM Pictures': 15,
         'Focus Film': 12,
         'Pana Film': 11,
         'ANS Productions': 10,
         'TMC': 10,
         'Yağmur Ajans': 9,
         'Mint Prodüksiyon': 9,
         'Ser Film': 9,
         'ARC Film': 9,
         'Base Productions': 8,
         'Sis Yapım': 8,
         'Acun Medya': 8,
         'Tims&B Productions': 8,
         'Bozdağ Film': 7,
         'Plato Film': 6,
         'Adam Film': 6,
         'Mia Yapım': 6,
         'Tükenmezkalem Film': 6,
   

# Besteci Dizi Sayısı

In [151]:
besteciler = []

In [154]:
feature_df['Besteci Dizi Sayısı'] = df['Besteci'].apply(lambda x: find_row_count(x, besteciler))

In [155]:
Counter(besteciler)

Counter({'Toygar Işıklı': 35,
         'Aytekin Ataş': 21,
         'Cem Tuncer': 19,
         'Alp Yenier': 18,
         'Zeynep Alasya': 16,
         'Cem Öget': 16,
         'Yıldıray Gürgen': 13,
         'Batuhan Fırat': 13,
         'Uğur Akyürek': 12,
         'Atakan Ilgazdağ': 12,
         'Ercüment Orkut': 12,
         'Alpay Göltekin': 12,
         'Tuna Hizmetli': 12,
         'Ender Gündüzlü': 12,
         'Güldiyar Tanrıdağlı': 12,
         'Mayki Murat Başaran': 11,
         'Metin Arıgül': 11,
         'Sunay Özgür': 10,
         'Aria': 9,
         'Ayşe Önder': 9,
         'Fırat Yükselir': 9,
         'Caner Özkan': 9,
         'Sertaç Özgümüş': 8,
         'Tamer Çıray': 8,
         'Engin Arslan': 8,
         'İnanç Şanver': 8,
         'Volkan Akmehmet': 8,
         'Kıraç': 7,
         'Aytuğ Yargıç': 7,
         'Cem Yıldız': 7,
         'Barış Aryay': 7,
         'Ümit Önder': 7,
         'Çağlar Haznedaroğlu': 7,
         'Uğur Ateş': 7,
         'Saki Çimen':

# Dizi Ödül Almış Mı ?

In [113]:
pantene_dizi_df = pd.read_csv('data/pantene_odullu_diziler.csv', index_col=0)
pantene_dizi_df.head()

,Yıl,Dizi,Kanal,Yapımcı
0,1992,Yıldızlar Gece Büyür,TRT 1,Altuğ Savaşal
1,1992,Olacak O Kadar,?,Levent Kırca
2,1992,Bizimkiler,?,Umur Bugay
3,1993,Bizimkiler,TRT 1,Umur Bugay
4,1993,Olacak O Kadar,?,Levent Kırca


In [114]:
feature_df['Ödüllü'] = df['Dizi Adı'].apply(lambda x: 1 if x in pantene_dizi_df['Dizi'].unique() else 0)
logging.debug('Dizi altın kelebek ödülü aldı mı özelliği bulundu.')

In [115]:
feature_df['Ödüllü'].value_counts()

Ödüllü
0    1199
1      53
Name: count, dtype: int64

# Kadın / Erkek Oyuncu Oranı

In [116]:
turk_dizi_oyuncular = pd.read_csv('data/turk_dizi_oyunculari.csv', index_col=0)
turk_dizi_oyuncular.head()

,Oyuncu Linki,Oyuncu Adı,Cinsiyet
0,https://tr.wikipedia.org//wiki/A%C3%A7elya_Akkoyun,Açelya Akkoyun,Kadın
1,https://tr.wikipedia.org//wiki/A%C3%A7elya_Devrim_Y%C4%B1lhan,Açelya Devrim Yılhan,Kadın
2,https://tr.wikipedia.org//wiki/A%C3%A7elya_Elmas,Açelya Elmas,Kadın
3,https://tr.wikipedia.org//wiki/A%C3%A7elya_%C3%96zcan,Açelya Özcan,Kadın
4,https://tr.wikipedia.org//wiki/A%C3%A7elya_Topalo%C4%9Flu,Açelya Topaloğlu,Kadın


In [117]:
def find_wm(text):
    count_w = 0
    count_m = 0
    
    texts = text.split(',')
    for t in texts:
        if t not in stopwords.words('turkish') and len(t) > 0 and t != '?':
            t = t.strip()   
            if t in turk_dizi_oyuncular['Oyuncu Adı'].unique():
                if turk_dizi_oyuncular[turk_dizi_oyuncular['Oyuncu Adı'] == t][['Cinsiyet']].values[0][0] == 'Kadın':
                    count_w += 1
                elif turk_dizi_oyuncular[turk_dizi_oyuncular['Oyuncu Adı'] == t][['Cinsiyet']].values[0][0] == 'Erkek':
                    count_m += 1
    if count_w + count_m > 0: 
        return count_w / (count_m + count_w)
    else:
        return 0

In [118]:
feature_df['Kadın Oyuncu Oranı'] = df['Başrol'].apply(find_wm)
logging.debug('Dizinin başrollerindeki kadın oyuncu oranı bulundu.')

# Gösterim Süresi

In [119]:
feature_df['duration'] = df['Gösterim süresi']
logging.debug('Dizinin gösterim süresi özelliği bulundu.')

# Kanal

In [120]:
df['Kanal'].value_counts().head(10)

Kanal
Diğer              235
TRT 1              160
Kanal D            155
atv                151
Star TV            141
Show TV            127
FOX                125
Samanyolu TV        30
TV8                 15
Kanal D,Star TV     11
Name: count, dtype: int64

In [121]:
feature_df['TRT 1'] = df['Kanal'].str.contains('TRT 1').astype(int)
feature_df['Kanal D'] = df['Kanal'].str.contains('Kanal D').astype(int)
feature_df['atv'] = df['Kanal'].str.contains('atv').astype(int)
feature_df['Star TV'] = df['Kanal'].str.contains('Star TV').astype(int)
feature_df['Show TV'] = df['Kanal'].str.contains('Show TV').astype(int)
feature_df['FOX'] = df['Kanal'].str.contains('FOX').astype(int)
feature_df['Samanyolu TV'] = df['Kanal'].str.contains('Samanyolu TV').astype(int)
feature_df['TV8'] = df['Kanal'].str.contains('TV8').astype(int)

In [122]:
logging.debug('Dizinin yayınlandığı kanal özelliği bulundu.')

# Sonuç

In [123]:
feature_df.head()

,Tarih,Dizi Adı,Dizi Adı Uzunluğu,Dizi Adındaki Kelime Sayısı,Dizi Adında Bağlaç,Dizi Adında Yer İsmi,Dizi Adında Özel İsim,in_turkey,out_turkey,act_count,book_to_movie,on_youtube,on_facebook,on_twitter,on_instagram,total_sm_accounts,Publication Day,summer_series,paid_channel,Aile,Aksiyon,Aşk,Bilim Kurgu,Dram,Gençlik,Gerilim,Komedi,Polisiye,Romantik,Tarihî,Senaristlerin Dizi Sayısı,Ödüllü Senarist Sayısı,Senaristlerin Aldığı Ödül Sayısı,Başrollerin Dizi Sayısı,Ödüllü Başrol Sayısı,Başrollerin Aldığı Ödül Sayısı,Yönetmenlerin Dizi Sayısı,Ödüllü Yönetmen Sayısı,Yönetmenlerin Aldığı Ödül Sayısı,Ödüllü,Kadın Oyuncu Oranı,duration,TRT 1,Kanal D,atv,Star TV,Show TV,FOX,Samanyolu TV,TV8
0,1974.00,Kaynanalar,10,1,0,0,0,1,0,8,0,0,0,0,0,0,Sunday,0,0,0,0,0,0,0,0,0,1,0,0,0,-1.00,0,0.00,1.00,0,0.00,1.00,0,0.00,0,0.60,58.00,1,1,0,0,0,0,0,0
1,1975.00,Aşk-ı Memnu,11,2,0,0,0,1,0,5,0,0,0,0,0,0,Saturday,0,0,1,0,0,0,1,0,0,0,0,0,0,1.00,0,0.00,1.00,0,0.00,1.00,0,0.00,1,0.80,58.00,1,0,0,0,0,0,0,0
2,1983.00,Kartallar Yüksek Uçar,21,3,0,0,0,1,0,5,0,0,0,0,0,0,Saturday,0,-1,0,0,0,0,0,0,0,0,1,0,0,1.00,0,0.00,1.00,0,0.00,1.00,0,0.00,0,0.40,58.00,0,0,0,0,0,0,0,0
3,1983.00,Üç İstanbul,11,2,0,0,0,1,0,5,0,0,0,0,0,0,?,-1,0,0,0,0,0,1,0,0,0,0,0,0,1.00,0,0.00,1.00,0,0.00,1.00,0,0.00,0,1.00,58.00,1,0,0,0,0,0,0,0
4,1984.00,Küçük Ağa,9,2,0,0,0,1,0,4,0,0,0,0,0,0,Sunday,0,0,0,1,0,0,1,0,0,0,0,0,1,1.00,0,0.00,1.00,0,0.00,1.00,0,0.00,1,0.25,58.00,1,0,0,0,0,0,0,0


In [156]:
processed_df = feature_df[['Tarih', 'Dizi Adı', 'Dizi Adı Uzunluğu', 'Dizi Adındaki Kelime Sayısı', 'Dizi Adında Bağlaç', 'Dizi Adında Yer İsmi', 'Dizi Adında Özel İsim',
                           'Aile', 'Aksiyon', 'Aşk', 'Bilim Kurgu', 'Dram', 'Gençlik', 'Gerilim', 'Komedi', 'Polisiye', 'Romantik', 'Tarihî',
                           'TRT 1', 'Kanal D', 'atv', 'Star TV', 'Show TV', 'FOX', 'Samanyolu TV', 'TV8',
                           'in_turkey', 'out_turkey', 'book_to_movie', 'act_count', 'paid_channel', 'Publication Day', 'summer_series', 'duration', 'Ödüllü',
                           'Senaristlerin Dizi Sayısı', 'Ödüllü Senarist Sayısı', 'Senaristlerin Aldığı Ödül Sayısı',
                           'Başrollerin Dizi Sayısı', 'Ödüllü Başrol Sayısı', 'Başrollerin Aldığı Ödül Sayısı',
                           'Yönetmenlerin Dizi Sayısı', 'Ödüllü Yönetmen Sayısı', 'Yönetmenlerin Aldığı Ödül Sayısı',
                           'Görüntü Yönetmenlerinin Dizi Sayısı', 'Yapım Şirketleri Dizi Sayısı', 'Besteci Dizi Sayısı',
                           'on_youtube', 'on_facebook', 'on_twitter', 'on_instagram', 'total_sm_accounts']]

In [157]:
processed_df['state'] = df['Durumu']

In [158]:
processed_df['state'] = processed_df['state'].apply(remove_parentheses)

In [159]:
processed_df['season'] = df['Sezon sayısı']

In [160]:
processed_df['season'] = processed_df['season'].apply(lambda x: 1 if x == '?' else int(x))

In [161]:
processed_df.head()

,Tarih,Dizi Adı,Dizi Adı Uzunluğu,Dizi Adındaki Kelime Sayısı,Dizi Adında Bağlaç,Dizi Adında Yer İsmi,Dizi Adında Özel İsim,Aile,Aksiyon,Aşk,Bilim Kurgu,Dram,Gençlik,Gerilim,Komedi,Polisiye,Romantik,Tarihî,TRT 1,Kanal D,atv,Star TV,Show TV,FOX,Samanyolu TV,TV8,in_turkey,out_turkey,book_to_movie,act_count,paid_channel,Publication Day,summer_series,duration,Ödüllü,Senaristlerin Dizi Sayısı,Ödüllü Senarist Sayısı,Senaristlerin Aldığı Ödül Sayısı,Başrollerin Dizi Sayısı,Ödüllü Başrol Sayısı,Başrollerin Aldığı Ödül Sayısı,Yönetmenlerin Dizi Sayısı,Ödüllü Yönetmen Sayısı,Yönetmenlerin Aldığı Ödül Sayısı,Görüntü Yönetmenlerinin Dizi Sayısı,Yapım Şirketleri Dizi Sayısı,Besteci Dizi Sayısı,on_youtube,on_facebook,on_twitter,on_instagram,total_sm_accounts,state,season
0,1974.00,Kaynanalar,10,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,8,0,Sunday,0,58.00,0,-1.00,0,0.00,1.00,0,0.00,1.00,0,0.00,-1.00,-1.00,-1.00,0,0,0,0,0,Sona erdi,14
1,1975.00,Aşk-ı Memnu,11,2,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,5,0,Saturday,0,58.00,1,1.00,0,0.00,1.00,0,0.00,1.00,0,0.00,-1.00,-1.00,-1.00,0,0,0,0,0,Sona erdi,1
2,1983.00,Kartallar Yüksek Uçar,21,3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,5,-1,Saturday,0,58.00,0,1.00,0,0.00,1.00,0,0.00,1.00,0,0.00,-1.00,-1.00,-1.00,0,0,0,0,0,Yayından kaldırıldı,1
3,1983.00,Üç İstanbul,11,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,5,0,?,-1,58.00,0,1.00,0,0.00,1.00,0,0.00,1.00,0,0.00,1.00,1.00,1.00,0,0,0,0,0,Sona erdi,1
4,1984.00,Küçük Ağa,9,2,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,4,0,Sunday,0,58.00,1,1.00,0,0.00,1.00,0,0.00,1.00,0,0.00,1.00,1.00,-1.00,0,0,0,0,0,Sona erdi,1


In [162]:
processed_df = processed_df[processed_df['season'] != '?']

In [163]:
processed_df.shape

(1252, 54)

In [164]:
processed_df['state'] = processed_df['state'].map({
    'Sona erdi': 'Final',
    '?': '?',
    'Sona erdi,': 'Final',
    'Sona erdi.': 'Final',
    'Ara verdi': '?',
    'Devam ediyor': '?',
    'Yayından kaldırıldı': 'Final',
    'Belirsiz': 'Final',
    'Yayından Kaldırıldı': 'Final',
    'Yayından kaldırıldı,': 'Final',
    'Final': 'Final',
    'Sezon arası': 'Final',
    'Yayımlandı': 'Final',
    'Yayınlandı': 'Final',
    'tabii,platformunda,Tozkoparan İskender: Gölge,olarak devam etmektedir.': 'Final',
    'Yayında': 'Final',
    '2.Sezon Çekimleri': 'Final',
    'Yakında': 'Final'
})

In [165]:
processed_df = processed_df[processed_df['state'] == 'Final']

In [166]:
processed_df.shape

(1213, 54)

In [167]:
processed_df['season'].unique()

array([14,  1,  5,  2,  4, 15, 10,  6,  7,  3,  8])

In [168]:
for index, row in processed_df.iterrows():
    if int(row['season']) == 1:
        processed_df.loc[index, 'state'] = 'Başarısız'
    elif int(row['season']) == 2:
        processed_df.loc[index, 'state'] = 'Kısmen Başarılı'
    elif int(row['season']) < 5:
        processed_df.loc[index, 'state'] = 'Başarılı'
    elif int(row['season']) < 10:
        processed_df.loc[index, 'state'] = 'Kısmen Olgun'
    elif int(row['season']) >= 10:
        processed_df.loc[index, 'state'] = 'Olgun'

In [169]:
logging.debug('Sezon sayısına göre işaretleme yapıldı.')

In [170]:
processed_df['state'].value_counts()

state
Başarısız          840
Kısmen Başarılı    214
Başarılı           114
Kısmen Olgun        39
Olgun                6
Name: count, dtype: int64

In [171]:
processed_df.drop('season', axis=1, inplace=True)

In [172]:
processed_df.tail()

,Tarih,Dizi Adı,Dizi Adı Uzunluğu,Dizi Adındaki Kelime Sayısı,Dizi Adında Bağlaç,Dizi Adında Yer İsmi,Dizi Adında Özel İsim,Aile,Aksiyon,Aşk,Bilim Kurgu,Dram,Gençlik,Gerilim,Komedi,Polisiye,Romantik,Tarihî,TRT 1,Kanal D,atv,Star TV,Show TV,FOX,Samanyolu TV,TV8,in_turkey,out_turkey,book_to_movie,act_count,paid_channel,Publication Day,summer_series,duration,Ödüllü,Senaristlerin Dizi Sayısı,Ödüllü Senarist Sayısı,Senaristlerin Aldığı Ödül Sayısı,Başrollerin Dizi Sayısı,Ödüllü Başrol Sayısı,Başrollerin Aldığı Ödül Sayısı,Yönetmenlerin Dizi Sayısı,Ödüllü Yönetmen Sayısı,Yönetmenlerin Aldığı Ödül Sayısı,Görüntü Yönetmenlerinin Dizi Sayısı,Yapım Şirketleri Dizi Sayısı,Besteci Dizi Sayısı,on_youtube,on_facebook,on_twitter,on_instagram,total_sm_accounts,state
1248,2023.00,Yeşil Vadi'nin Kızı,19,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,3,0,Monday,1,70.00,0,1.40,0,0.00,2.67,0,0.00,1.00,0,0.00,3.50,1.00,1.00,0,0,0,0,0,Başarısız
1249,2023.00,Yüz Yıllık Mucize,17,3,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,2,0,Thursday,0,135.00,0,12.00,1,4.00,6.50,1,1.50,11.00,0,0.00,-1.00,14.00,18.00,0,0,1,1,2,Başarısız
1250,2023.00,Ya Çok Seversen,15,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,2,0,Thursday,1,130.00,1,4.00,0,0.00,6.50,1,2.50,4.50,1,1.75,-1.00,51.00,10.00,1,0,1,1,3,Başarısız
1252,2023.00,Şebeke,6,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4,-1,Sunday,0,140.00,0,2.50,0,0.00,5.25,0,0.00,2.00,0,0.00,-1.00,1.00,-1.00,0,0,0,0,0,Başarısız
1255,2023.00,"Sen, Ben, O!",12,6,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,-1,Tuesday,0,15.00,0,1.00,0,0.00,1.50,0,0.00,2.00,0,0.00,-1.00,1.00,1.00,0,0,0,0,0,Başarısız


In [173]:
processed_df = processed_df.reset_index(drop=True)

In [174]:
logging.debug('Özellik veri seti kaydedildi.')

In [175]:
processed_df.to_csv('data/processed.csv')